<a href="https://colab.research.google.com/github/CD-AC/DataScience-Sales_Prediction/blob/main/Prediction_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from prophet import Prophet
import datetime

In [4]:
# Cargar los datos
store_data_path = ("/content/drive/MyDrive/sales_prediction/store.csv")
train_data_path = ("/content/drive/MyDrive/sales_prediction/train.csv")

store_data = pd.read_csv(store_data_path)
train_data = pd.read_csv(train_data_path)

<ipython-input-4-264ee6ef61e7>:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(train_data_path)


In [5]:
# Ingeniería de características
train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data['Year'] = train_data['Date'].dt.year
train_data['Month'] = train_data['Date'].dt.month
train_data['Day'] = train_data['Date'].dt.day
train_data['WeekOfYear'] = train_data['Date'].dt.isocalendar().week
train_data = train_data.merge(store_data, on='Store')

In [6]:
# Seleccionar características relevantes para el modelo
features = ['Store', 'DayOfWeek', 'Promo', 'Year', 'Month', 'Day', 'WeekOfYear',
            'CompetitionDistance', 'Promo2']
target = 'Sales'

In [7]:
# Imputar valores faltantes
train_data['CompetitionDistance'].fillna(train_data['CompetitionDistance'].mean(), inplace=True)
train_data['Promo2SinceWeek'].fillna(0, inplace=True)
train_data['Promo2SinceYear'].fillna(0, inplace=True)

In [8]:
# Escalar los datos
scaler = MinMaxScaler()
train_data[features] = scaler.fit_transform(train_data[features])

In [10]:
# Preparar los datos para el modelo LSTM
X = train_data[features].values
y = train_data[target].values
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [11]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Crear el modelo LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(LSTM(50, return_sequences=False))
model_lstm.add(Dense(25))
model_lstm.add(Dense(1))

model_lstm.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# Entrenar el modelo LSTM
history_lstm = model_lstm.fit(X_train, y_train, batch_size=1, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
813767/813767 [==============================] - 3496s 4ms/step - loss: 7860182.5000 - val_loss: 7362662.5000
Epoch 2/10
813767/813767 [==============================] - 3542s 4ms/step - loss: 7150414.0000 - val_loss: 6716389.5000
Epoch 3/10
  1841/813767 [..............................] - ETA: 58:42 - loss: 6732152.0000

KeyboardInterrupt: 

In [14]:
# Predecir las ventas con LSTM
predictions_lstm = model_lstm.predict(X_test)
predictions_lstm = scaler.inverse_transform(predictions_lstm)

6358/6358 [==============================] - 16s 2ms/step


ValueError: non-broadcastable output operand with shape (203442,1) doesn't match the broadcast shape (203442,9)

In [15]:
# Preparar los datos para Prophet
sales_train_df = train_data[['Date', 'Sales']]
sales_train_df.columns = ['ds', 'y']

In [ ]:
# Entrenar el modelo Prophet
model_prophet = Prophet()
model_prophet.fit(sales_train_df)
future = model_prophet.make_future_dataframe(periods=365)
forecast = model_prophet.predict(future)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvtnxz1x5/3nlbfq3_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvtnxz1x5/qdojx88t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=50091', 'data', 'file=/tmp/tmpvtnxz1x5/3nlbfq3_.json', 'init=/tmp/tmpvtnxz1x5/qdojx88t.json', 'output', 'file=/tmp/tmpvtnxz1x5/prophet_model5qfrbfru/prophet_model-20240626004610.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:46:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


In [ ]:
# Evaluar el rendimiento de Prophet en el conjunto de prueba
y_test_prophet = sales_train_df[sales_train_df['ds'].isin(train_data['Date'][X_test[:,:,0].shape[0]:])]['y']
y_pred_prophet = forecast['yhat'][-y_test_prophet.shape[0]:]

In [ ]:
# Visualizar los resultados
plt.figure(figsize=(12, 6))
plt.plot(y_test, color='blue', label='Ventas Reales')
plt.plot(predictions_lstm, color='red', label='Ventas Predichas por LSTM')
plt.plot(y_pred_prophet.values, color='green', label='Ventas Predichas por Prophet')
plt.title('Predicción de Ventas')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.show()

In [ ]:
# Evaluación y comparación de modelos
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_lstm = mean_absolute_error(y_test, predictions_lstm)
mse_lstm = mean_squared_error(y_test, predictions_lstm)
mae_prophet = mean_absolute_error(y_test_prophet, y_pred_prophet)
mse_prophet = mean_squared_error(y_test_prophet, y_pred_prophet)

print(f"MAE LSTM: {mae_lstm}")
print(f"MSE LSTM: {mse_lstm}")
print(f"MAE Prophet: {mae_prophet}")
print(f"MSE Prophet: {mse_prophet}")